02で前処理をしたデータの読み込みと03で学習したモデルを使って推論を行うためのnotebookです。  

## 必要なライブラリのimport

In [22]:
import numpy as np
import pandas as pd
import os
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_error
import pickle

In [23]:
# windows
if os.name == 'nt':
    path = '../../../data/processed/'
    modelpath = '../../src/models/nakagawa/'
else:
    if 'KAGGLE_DATA_PROXY_TOKEN' in os.environ.keys():
        path = '/kaggle/input/processed/'


# FIXME:
PATHを修正すること！！！

In [24]:
# FIXME: PATH
# train_X_path = os.path.join(path,'processed20240618_train_X.csv')
# test_X_path = os.path.join(path,'processed20240618_test_X.csv')
# train_y_path = os.path.join(path,'processed20240618_train_y.csv')
# test_y_path = os.path.join(path,'processed20240618_test_y.csv')
train_path = os.path.join(path,'processed20240619-2_train.csv')
test_path = os.path.join(path,'processed20240619-2_test.csv')

In [25]:
# データの読み込み
# X_test = pd.read_csv(test_X_path)
# y_test = pd.read_csv(test_y_path)
train= pd.read_csv(train_path)
test = pd.read_csv(test_path)

# del X_test['target']


In [26]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,...,city_id_purchase_amount_std,category_1_installments_mean,category_1_installments_min,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean
0,2017/4/1,C_ID_0ab67a22ab,3,3,1,306.0,24.0,13.0,0.541667,0.458333,...,0.182864,1.666667,1.666667,1.666667,NaN,1.004401,1.000000,0.140001,-6.388255,1.120009
1,2017/1/1,C_ID_130fd0cbdd,2,3,0,396.0,1.0,1.0,1.000000,0.000000,...,0.014353,1.142857,1.000000,1.285714,0.202031,1.020192,-inf,8.121995,0.691844,0.902444
2,2017/8/1,C_ID_b709037bc5,5,1,1,184.0,4.0,0.0,0.000000,0.000000,...,1.151811,5.500000,1.000000,10.000000,6.363961,1.002210,0.500000,0.033132,0.358015,0.066264
3,2017/12/1,C_ID_d27d835a9f,2,1,0,62.0,NaN,NaN,NaN,NaN,...,0.083170,6.944444,1.888889,12.000000,7.149635,NaN,NaN,NaN,NaN,NaN
4,2015/12/1,C_ID_2b5e3df5c2,5,1,1,793.0,23.0,0.0,0.000000,0.000000,...,1.288398,0.833333,0.833333,0.833333,NaN,1.005639,3.833333,-0.775375,39.511246,-2.972271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017/10/1,C_ID_7a239d2eda,2,2,0,123.0,3.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123619,2017/9/1,C_ID_75ace375ae,3,1,1,153.0,NaN,NaN,NaN,NaN,...,0.097762,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
123620,2016/9/1,C_ID_21d56d950c,5,1,1,518.0,1.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123621,2017/6/1,C_ID_6c46fc5a9d,2,1,0,245.0,19.0,0.0,0.000000,0.000000,...,0.062056,1.300000,0.600000,2.000000,0.989949,1.003513,1.091954,0.306193,0.999174,0.969611


## 保存されたモデルを使って予測を行う

In [27]:
def predict_model(test_X,model_name,test_y=False,flg=False):
    n_folds = 5
    predictions = np.zeros(len(test_X))
    features = [c for c in test_X.columns if c not in ['card_id', 'first_active_month']]

    for fold_ in range(n_folds):
        # モデルを読み込む
        with open(f'{modelpath}{model_name}/{model_name}_{fold_}.pkl', 'rb') as f:
            clf = pickle.load(f)

        # モデルで予測を行う
        fold_predictions = clf.predict(test_X[features])

        # 予測結果を足し合わせる
        predictions += fold_predictions

    # 平均を取る
    predictions /= n_folds
    if flg == False:
        # RMSEを計算する（テストデータに対して真の値がある場合）
        # y_testが存在する場合に限り
        score = root_mean_squared_error(test_y, predictions)
        print(f'RMSE={score}')
    return predictions

# テストデータを予測する 
## ①train_Xモデルを用いてtestデータを予測してRMSEを出す
## (モデルネーム確認すること！！)


In [28]:
# print(predict_model(test_X=X_test,test_y=y_test,model_name='train_X_model_20240618'))

## ②train_allモデルを用いてtestデータでの提出用の予測値を出す

### 提出用ファイルの作成　(モデルネーム確認すること！！)

In [29]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predict_model(test_X=test,model_name='train_lgb_model_20240619-2',test_y=False,flg=True)
sub_df.to_csv("submit20240619-2.csv", index=False)